# System Info

References: [thepythoncode.com](https://www.thepythoncode.com/article/get-hardware-system-information-python)

Open a shell an enter install psutil

```
pip3 install psutil
```

Let's make a function that converts large number of bytes into a scaled format (e.g in kilo, mega, giga, etc.):

In [157]:
import psutil
import platform
import pandas as pd
from datetime import datetime

def get_size(bytes, suffix="B"):
    """
    Scale bytes to its proper format
    e.g:
        1253656 => '1.20MB'
        1253656678 => '1.17GB'
    """
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor


## System Information

We gonna need platform module here:

### Platform information

In [158]:
def get_sysinfo():
    """
    Get some system information from platform module
    """
    # Get systeminfo
    uname = platform.uname()

    # Boot Time
    boot_time_timestamp = psutil.boot_time()
    boot_time_formated  = datetime.fromtimestamp(boot_time_timestamp)
    
    table_data = [{
        'System'   : uname.system,
        'Node Name': uname.node,
        'Release'  : uname.release,
        'Version'  : uname.version,
        'Machine'  : uname.machine,
        'Processor': uname.processor,
        'Boot Time': boot_time_formated,
    }]

    return pd.DataFrame(data = table_data)

### CPU information

In [159]:
def get_cpu_info(detailed = False):
    """
    Get some CPU information, such as total number of cores, usage, etc
    """
    cpufreq = psutil.cpu_freq()

    if detailed:
        table_data = [{
            "Physical cores"    : psutil.cpu_count(logical=False),
            "Total cores"       : psutil.cpu_count(logical=True),
            "Max Frequency"     : f"{cpufreq.max:.0f}Mhz",
            "Min Frequency"     : f"{cpufreq.min:.0f}Mhz",
            "Current Frequency" : f"{cpufreq.current:.0f}Mhz",
            "Total CPU Usage"  : f"{psutil.cpu_percent():.2f}%",
        }]
        # CPU usage
        print("CPU Usage Per Core:")
        for i, percentage in enumerate(psutil.cpu_percent(percpu=True, interval=1)):
            print(f"Core {i}:\t{percentage}%")
    else:
        table_data = [{
            "Physical cores"   : psutil.cpu_count(logical=False),
            "Total cores"      : psutil.cpu_count(logical=True),
            "Current Frequency": f"{cpufreq.current:.0f}Mhz",
            "Total CPU Usage"  : f"{psutil.cpu_percent():.2f}%",
        }]      

    return pd.DataFrame(data = table_data)

### Memory information

In [160]:
def get_memory_info():
    """
    Get memory information
    """
    # get the memory details
    svmem = psutil.virtual_memory()
    table_data = [{
        "Total"     : f"{get_size(svmem.total)}",
        "Available" : f"{get_size(svmem.available)}",
        "Used"      : f"{get_size(svmem.used)}",
        "Percentage": f"{svmem.percent}%",
    }]      

    return pd.DataFrame(data = table_data)      

def get_swap_info():
    """
    Get swap information
    """
    # get the swap memory details (if exists)
    swap = psutil.swap_memory()
    table_data = [{
        "Total"     : f"{get_size(swap.total)}",
        "Free"      : f"{get_size(swap.free)}",
        "Used"      : f"{get_size(swap.used)}",
        "Percentage": f"{swap.percent}%",
    }]      

    return pd.DataFrame(data = table_data)    

### Disk information

In [161]:
def get_disk_info():
    """
    Get disk information
    """
    partitions = psutil.disk_partitions()
    table_data = []
    for partition in partitions:
        try:
            partition_usage = psutil.disk_usage(partition.mountpoint)
        except PermissionError:
            # this can be catched due to the disk that
            # isn't ready
            continue

        thisdict = {  
            "Device"          : f"{partition.device}",
            "Mountpoint"      : f"{partition.mountpoint}",
            "File system type": f"{partition.fstype}",
            "Total Size"      : f"{get_size(partition_usage.total)}",
            "Used"            : f"{get_size(partition_usage.used)}",
            "Free"            : f"{get_size(partition_usage.free)}",
            "Percentage"      : f"{partition_usage.percent}%",
        }
        table_data.append(thisdict)

    return pd.DataFrame(data = table_data)    

### Network information

In [163]:
def get_net_info():
    """
    Get network information
    """
    # get all network interfaces (virtual and physical)
    table_data = []
    if_addrs = psutil.net_if_addrs()
    for interface_name, interface_addresses in if_addrs.items():
        for address in interface_addresses:
            if str(address.family) == 'AddressFamily.AF_INET':
                thisdict = {  
                    "Interface"   : f"{interface_name}",
                    "IP Address"  : f"{address.address}",
                    "Netmask"     : f"{address.netmask}",
                    "Broadcast IP": f"{address.broadcast}",
                }
            elif str(address.family) == 'AddressFamily.AF_PACKET':
                thisdict = {  
                    "Interface"    : f"{interface_name}",
                    "MAC Address"  : f"{address.address}",
                    "Netmask"      : f"{address.netmask}",
                    "Broadcast MAC": f"{address.broadcast}",
                }
        table_data.append(thisdict)

    return pd.DataFrame(data = table_data)

### Print all information

In [165]:
totallength = 110

# let's print system information
displaystr = "System Information"
sidelengt = ((totallength - len(displaystr)) / 2)
print("="*int(sidelengt), displaystr, "="*int(sidelengt))
display(get_sysinfo())

# let's print CPU information
displaystr = "CPU Information"
sidelengt = ((totallength - len(displaystr)) / 2)
print("="*int(sidelengt), displaystr, "="*int(sidelengt))
display(get_cpu_info())

# let's print memory information
displaystr = "Memory Information"
sidelengt = ((totallength - len(displaystr)) / 2)
print("="*int(sidelengt), displaystr, "="*int(sidelengt))
display(get_memory_info())

# let's print swap information
displaystr = "Swap Information"
sidelengt = ((totallength - len(displaystr)) / 2)
print("="*int(sidelengt), displaystr, "="*int(sidelengt))
display(get_swap_info())

# let's print Disk information
displaystr = "Disk Information"
sidelengt = ((totallength - len(displaystr)) / 2)
print("="*int(sidelengt), displaystr, "="*int(sidelengt))
display(get_disk_info())

# get IO statistics since boot
disk_io = psutil.disk_io_counters()
print(f"Total read : {get_size(disk_io.read_bytes)}")
print(f"Total write: {get_size(disk_io.write_bytes)}")

# let's print Network information
displaystr = "Network Information"
sidelengt = ((totallength - len(displaystr)) / 2)
print("="*int(sidelengt), displaystr, "="*int(sidelengt))
display(get_net_info())

# get IO statistics since boot
net_io = psutil.net_io_counters()
print(f"Total Bytes Sent    : {get_size(net_io.bytes_sent)}")
print(f"Total Bytes Received: {get_size(net_io.bytes_recv)}")

displaystr = "END"
sidelengt = ((totallength - len(displaystr)) / 2)
print("="*int(sidelengt), displaystr, "="*int(sidelengt))

============================================== System Information ==============================================


,System,Node Name,Release,Version,Machine,Processor,Boot Time
0,Darwin,tinus-imac.home,20.3.0,Darwin Kernel Version 20.3.0: Thu Jan 21 00:07...,x86_64,i386,2021-05-20 07:54:08


=============================================== CPU Information ===============================================


,Physical cores,Total cores,Current Frequency,Total CPU Usage
0,4,8,3600Mhz,39.00%


============================================== Memory Information ==============================================


...

=============================================== Disk Information ===============================================


,Device,Mountpoint,File system type,Total Size,Used,Free,Percentage
0,/dev/disk1s5s1,/,apfs,465.63GB,14.02GB,95.80GB,12.8%
1,/dev/disk1s4,/System/Volumes/VM,apfs,465.63GB,3.00GB,95.80GB,3.0%
2,/dev/disk1s2,/System/Volumes/Preboot,apfs,465.63GB,412.00MB,95.80GB,0.4%
3,/dev/disk1s6,/System/Volumes/Update,apfs,465.63GB,4.07MB,95.80GB,0.0%
4,/dev/disk1s1,/System/Volumes/Data,apfs,465.63GB,351.67GB,95.80GB,78.6%


Total read : 22.75GB
Total write: 935.33GB
============================================= Network Information =============================================


,Interface,IP Address,Netmask,Broadcast IP
0,lo0,127.0.0.1,255.0.0.0,None
1,en1,192.168.1.112,255.255.255.0,192.168.1.255
2,bridge100,192.168.64.1,255.255.255.0,192.168.64.255
3,bridge100,192.168.64.1,255.255.255.0,192.168.64.255
4,bridge100,192.168.64.1,255.255.255.0,192.168.64.255
5,bridge100,192.168.64.1,255.255.255.0,192.168.64.255
6,bridge100,192.168.64.1,255.255.255.0,192.168.64.255
7,bridge100,192.168.64.1,255.255.255.0,192.168.64.255
8,bridge100,192.168.64.1,255.255.255.0,192.168.64.255
9,bridge100,192.168.64.1,255.255.255.0,192.168.64.255


Total Bytes Sent    : 152.78MB
Total Bytes Received: 419.16MB
===================================================== END =====================================================
